# Initialization

In [1]:
using CSV, DataFrames, JuMP, HiGHS, LinearAlgebra, DataStructures, Statistics
baltic_demand = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/Demand_Baltic.csv", delim='\t'))
ports = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/ports.csv", delim='\t'))
#ports.PortCallCostFixed = coalesce.(ports.PortCallCostFixed, 0.0)
distances = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/dist_dense.csv", delim='\t'))
baltic_fleet = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_Baltic.csv", delim='\t'))
fleet_data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_data.csv", delim='\t'))
; # Don't print output

### Services:

The following array contains manual generated services to which the column generation will be applied.

In [2]:
# services = [["Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#             ["Feeder_800" "RUKGD" "FIKTK" "RUKGD"],
#             ["Feeder_450" "DEBRV" "NOSVG" "DEBRV"],
#             ["Feeder_450" "NOAES" "NOBGO" "NOAES"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#             ["Feeder_800" "DEBRV" "SEGOT" "DKAAR" "DEBRV"],
#             ["Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
#             ["Feeder_450" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "SEGOT"],
#             ["Feeder_450" "DEBRV" "NOAES" "DEBRV" "RULED" "DEBRV"],
#             ["Feeder_450" "DEBRV" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "DEBRV"],
#             ["Feeder_800" "PLGDY" "RUKGD" "RULED" "FIKTK" "FIRAU" "PLGDY"],
#             ["Feeder_450" "PLGDY" "RUKGD" "FIKTK" "RULED" "FIRAU" "PLGDY"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "NOKRS"],
#             ["Feeder_800" "NOAES" "NOBGO" "NOSVG" "NOKRS" "SEGOT" "DKAAR" "SEGOT" "NOKRS" "NOSVG" "NOBGO" "NOAES"]]

# services = [["Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#                 ["Feeder_800" "NOKRS" "NOSVG" "NOBGO" "NOAES" "NOKRS"],
#                 ["Feeder_800" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#                 ["Feeder_800" "DKAAR" "PLGDY" "RUKGD" "DKAAR"],
#                 ["Feeder_800" "SEGOT" "FIRAU" "FIKTK" "RULED" "SEGOT"]]

services = [["Feeder" "DEBRV" "DKAAR" "SEGOT" "FIRAU" "DEBRV"],
            ["Feeder" "NOKRS" "DKAAR" "SEGOT" "NOBGO" "NOKRS"]]



# Initialize an empty Set to store unique ports
service_ports = []
# Iterate through the subarrays and add the unique ports to the set
for service in services
    for port in service[2:end]
        push!(service_ports, port)
    end
end
service_ports = unique(service_ports);


In [3]:
# List of ports that have intersecting service
intersect_ports = []
for i in 1:length(services)
    for j in i+1:length(services)
        push!(intersect_ports, intersect([[row[2:end]...] for row in services][i], [[row[2:end]...] for row in services][j]))
    end
end
intersect_ports = collect(Set(i for j in intersect_ports for i in j)); # Collecting unique transshipment nodes

# Adding transshipment nodes where services intersect
services_w_transshipment = []
transshipment_nodes = []
i = 1
for service in services
    updated_service = []
    for port in service
        if port in intersect_ports
            push!(updated_service, port * string(i)) # vi behøves i realiteten ikke at update vores serivces
            push!(transshipment_nodes, port * string(i))
        else
            push!(updated_service, port)
        end
    end
    push!(services_w_transshipment, updated_service)
    i += 1
end
transshipment_nodes = unique(transshipment_nodes);

In [27]:
# Source and terminal adjacency list
s_t_adj = []
for port in service_ports
    for node in transshipment_nodes
        if port == node[1:5]
            push!(s_t_adj, [string(port, "s"), node, parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
            push!(s_t_adj, [node, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
        end
    end
    if port ∉ intersect_ports
        push!(s_t_adj, [string(port, "s"), port, parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
        push!(s_t_adj, [port, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
    end
end

# Transshipment nodes adjacency list
transshipment_adj = []
for node in intersect_ports, i in transshipment_nodes, j in transshipment_nodes
    if j!=i && contains(i, node) && contains(j, node)
        push!(transshipment_adj, [i, j, parse.(Float64, ports[ports.UNLocode .== node, 10][1])])
    end
end

# Service ports adjacency list UNDERSØG OMKOSTNINGERNE MED STEFAN
ports_adj = []
test = []
for service in 1:length(services_w_transshipment)
    for port in 3:length(services_w_transshipment[service])
        push!(ports_adj, [services_w_transshipment[service][port-1], services_w_transshipment[service][port], parse.(Float64, ports[ports.UNLocode .== services_w_transshipment[service][port][1:5], 9][1]) + ports[ports.UNLocode .== services_w_transshipment[service][port][1:5], 12][1]])
    end
end

# Penalty edge from every source to every terminal
penalty_adj = []
for porti in 1:length(service_ports)
    for portj in 1:length(service_ports)
        if service_ports[porti] != service_ports[portj]
            push!(penalty_adj, [string(service_ports[porti], "s"), string(service_ports[portj], "t"), 50000.0])
        end
    end
    push!(penalty_adj, [string(service_ports[porti], "t"), string(service_ports[porti], "t"), 0.0])
end


edges = vcat(s_t_adj, transshipment_adj, ports_adj, penalty_adj);

In [ ]:
# For hver node operetter vi Ns (source) og Nt (terminal) samt den originiale N. Her har vi følgende adj list:
    # Ns -> N med load cost og N -> Nt med disch cost.

# Dem fra Ns skal gå hen i naboernes normale

# Vi skal også tilføje transshipment noderne

In [29]:
# Create an empty adjacency list
adjacency_list = Dict{String, Vector{Tuple{String, Float64}}}()  # Use Tuple to store (target, VC)

# Populate the adjacency list
for edge in edges
    from_node, to_node, VC = edge  # Extract source, target, and VC
    if to_node ∈ unique([[row[2:2]...] for row in penalty_adj])
        push!(adjacency_list[to_node], (to_node, VC))
    elseif haskey(adjacency_list, from_node)
        push!(adjacency_list[from_node], (to_node, VC))  # Add the target and VC to the source's list
    else
        adjacency_list[from_node] = [(to_node, VC)]  # Create a new entry for the source
    end
end
adjacency_list

Dict{String, Vector{Tuple{String, Float64}}} with 20 entries:
  "DKAAR2" => [("DKAARt", 429.0), ("DKAAR1", 203.0), ("SEGOT2", 260.0)]
  "FIRAUt" => [("FIRAUt", 0.0)]
  "NOKRS"  => [("NOKRSt", 141.0), ("DKAAR2", 436.0)]
  "DKAARs" => [("DKAAR1", 429.0), ("DKAAR2", 429.0), ("DEBRVt", 50000.0), ("SEG…
  "SEGOT1" => [("SEGOTt", 247.0), ("SEGOT2", 143.0), ("FIRAU", 323.0)]
  "NOBGOs" => [("NOBGO", 365.0), ("DEBRVt", 50000.0), ("DKAARt", 50000.0), ("SE…
  "NOBGO"  => [("NOBGOt", 365.0), ("NOKRS", 321.0)]
  "DKAARt" => [("DKAARt", 0.0)]
  "SEGOTs" => [("SEGOT1", 247.0), ("SEGOT2", 247.0), ("DEBRVt", 50000.0), ("DKA…
  "SEGOT2" => [("SEGOTt", 247.0), ("SEGOT1", 143.0), ("NOBGO", 484.0)]
  "FIRAU"  => [("FIRAUt", 196.0), ("DEBRV", 213.0)]
  "DEBRVt" => [("DEBRVt", 0.0)]
  "NOKRSt" => [("NOKRSt", 0.0)]
  "DEBRV"  => [("DEBRVt", 199.0), ("DKAAR1", 436.0)]
  "NOKRSs" => [("NOKRS", 141.0), ("DEBRVt", 50000.0), ("DKAARt", 50000.0), ("SE…
  "NOBGOt" => [("NOBGOt", 0.0)]
  "DEBRVs" => [("DEBRV", 199.0

In [39]:
source = "DEBRVs"
destination = "FIRAUt"


dist = Dict{Any, Float64}()
prev = Dict{Any, Any}()
Q = Set(keys(adjacency_list))

for v in Q
    dist[v] = Inf
    prev[v] = nothing
end

dist[source] = 0.0


while !isempty(Q)

    # Finds the minimum distance in Q
    min_value = Inf
    min_key = string()
    for i in Q
        if dist[i] <= min_value
            min_value = dist[i]
            min_key = i
        end
    end
    u = min_key

    if dist[u] == Inf
        break
    end
    if u == destination
        break
    end
    
    delete!(Q, u)

    for v in adjacency_list[u]
        if haskey(adjacency_list, v[1])
            alt = dist[u] + v[2]
            if alt < dist[v[1]]
                dist[v[1]] = alt
                prev[v[1]] = u
            end
        else
            break
        end
    end
end

S = []
u = destination

while prev[u] != nothing
    push!(S, u)
    u = prev[u]
end

println(reverse(S))
println(dist[destination])
# return S
# end


Any["DEBRV", "DKAAR1", "SEGOT1", "FIRAU", "FIRAUt"]
1414.0


In [36]:
dist

Dict{Any, Float64} with 20 entries:
  "DKAAR2" => 838.0
  "FIRAUt" => 1414.0
  "NOKRS"  => Inf
  "DKAARs" => Inf
  "SEGOT1" => 895.0
  "NOBGOs" => Inf
  "NOBGO"  => 1522.0
  "DKAARt" => 1064.0
  "SEGOTs" => Inf
  "FIRAU"  => 1218.0
  "SEGOT2" => 1038.0
  "DEBRVt" => 398.0
  "NOKRSt" => 50000.0
  "DEBRV"  => 199.0
  "NOKRSs" => Inf
  "NOBGOt" => 50000.0
  "DEBRVs" => 0.0
  "SEGOTt" => 1142.0
  "FIRAUs" => Inf
  "DKAAR1" => 635.0

In [137]:
u = "DEBRVs"

"DEBRVs"

In [161]:
for v in Q
    print(v)
end

DKAAR2NOKRSDKAARsSEGOT1NOBGOsNOBGOSEGOTsSEGOT2FIRAUDEBRVNOKRSsFIRAUsDKAAR1

In [145]:

#function Dijkstra(Graph, origin, destination)

#dist = Dict((v => Inf) for v in keys(adjacency_list))
#prev = Dict((v => "") for v in keys(adjacency_list))
#dist[start] = 0.0

dist = Dict{Any, Float64}()
prev = Dict{Any, Any}()
Q = Set(keys(adjacency_list))

for v in Q
    dist[v] = Inf
    prev[v] = nothing
end

for v in adjacency_list[u]
    print(v, "sut")
    if haskey(adjacency_list, v[1])
        print(v, "hey")
        alt = dist[u] + v[2]
        if alt < dist[v[1]]
            dist[v[1]] = alt
            prev[v[1]] = u
        end
    else
        break
    end
end

KeyError: KeyError: key "DKAARt" not found

In [139]:
haskey(adjacency_list,"DKAARt")

false

In [15]:
u

"DEBRVs"

In [37]:
adjacency_list

Dict{String, Vector{Tuple{String, Float64}}} with 20 entries:
  "DKAAR2" => [("DKAAR2", 0.0), ("DKAARt", 429.0), ("DKAAR1", 1061.0), ("SEGOT2…
  "FIRAUt" => [("FIRAUt", 0.0)]
  "NOKRS"  => [("NOKRS", 0.0), ("NOKRSt", 141.0), ("DKAAR2", 436.0)]
  "DKAARs" => [("DKAAR1", 429.0), ("DKAAR2", 429.0), ("DEBRVt", 1000.0), ("SEGO…
  "SEGOT1" => [("SEGOT1", 0.0), ("SEGOTt", 247.0), ("SEGOT2", 637.0), ("FIRAU",…
  "NOBGOs" => [("NOBGO", 365.0), ("DEBRVt", 1000.0), ("DKAARt", 1000.0), ("SEGO…
  "NOBGO"  => [("NOBGO", 0.0), ("NOBGOt", 365.0), ("NOKRS", 321.0)]
  "DKAARt" => [("DKAARt", 0.0)]
  "SEGOTs" => [("SEGOT1", 247.0), ("SEGOT2", 247.0), ("DEBRVt", 1000.0), ("DKAA…
  "SEGOT2" => [("SEGOT2", 0.0), ("SEGOTt", 247.0), ("SEGOT1", 637.0), ("NOBGO",…
  "FIRAU"  => [("FIRAU", 0.0), ("FIRAUt", 196.0), ("DEBRV", 213.0)]
  "DEBRVt" => [("DEBRVt", 0.0)]
  "NOKRSt" => [("NOKRSt", 0.0)]
  "DEBRV"  => [("DEBRV", 0.0), ("DEBRVt", 199.0), ("DKAAR1", 436.0)]
  "NOKRSs" => [("NOKRS", 141.0), ("DEBRVt", 1000.0

In [46]:
start = "DEBRVs"
destination = "DKAARt"
# function Dijkstra(Graph, origin, destination)
dist = Dict((v => Inf) for v in keys(adjacency_list))
prev = Dict((v => string() for v in keys(adjacency_list)))
dist[start] = 0.0
Q = Set(keys(adjacency_list))

for v in adjacency_list[u]
    println(v)
    alt = dist[u] + v[2]
    println(alt)
    if alt < dist[v[1]]
        dist[v[1]] = alt
        println(dist[v[1]])
        prev[v[1]] = u
        println(prev[v[1]])
    end
end

("DEBRV", 199.0)
199.0
199.0
DEBRVs
("DKAARt", 1000.0)
1000.0
1000.0
DEBRVs
("SEGOTt", 1000.0)
1000.0
1000.0
DEBRVs
("FIRAUt", 1000.0)
1000.0
1000.0
DEBRVs
("NOKRSt", 1000.0)
1000.0
1000.0
DEBRVs
("NOBGOt", 1000.0)
1000.0
1000.0
DEBRVs


In [45]:
prev["DEBRVt"]

String

In [18]:
dist

Dict{String, Float64} with 20 entries:
  "DEBRVt" => Inf
  "DKAAR2" => Inf
  "NOKRSt" => 1000.0
  "DEBRV"  => 199.0
  "FIRAUt" => 1000.0
  "NOKRS"  => Inf
  "DKAARs" => Inf
  "SEGOT1" => Inf
  "NOBGOs" => Inf
  "NOBGO"  => Inf
  "NOKRSs" => Inf
  "NOBGOt" => 1000.0
  "DEBRVs" => 0.0
  "SEGOTt" => 1000.0
  "FIRAUs" => Inf
  "DKAARt" => 1000.0
  "DKAAR1" => Inf
  "SEGOTs" => Inf
  "SEGOT2" => Inf
  "FIRAU"  => Inf

In [29]:
for v in adjacency_list[u]
    print(v)
    alt = dist[u][1] + v[2]

    if alt <= dist[v[1]]
        print(alt)

        println(string(alt), dist[v[1]])
        print(sut)
        dist[v[1]] = alt
        prev[v[1]] = u
    end
end

("DEBRV", 199.0)("DKAARt", 1000.0)("SEGOTt", 1000.0)("FIRAUt", 1000.0)("NOKRSt", 1000.0)("NOBGOt", 1000.0)

In [31]:
dist[("DEBRV", 199.0)[1]]

199.0

In [25]:
dist[u][1]+("DEBRV", 199.0)[2]

199.0

In [ ]:
u = findmin(dist)[2]

start = "DEBRVs"
destination = "DKAARt"


In [ ]:
Dijkstra(adjacency_list, "DEBRVs", "DKAARt")

In [ ]:
delete!(Q, u)

In [ ]:
Q

Berfore the master problem is formulated, the $c_{y}$ constants are calculated which correspond to the fixed port costs (more might be added later).

Moreover, the sum of all paths must be equal to $d_{k}$, in order to fulfill the demand. However later on, a new expensive penalty edge is added which penalizes all the containers that are not delivered to the commodity port.

As the decision variables, $x_{i,j}$ represent the edges in the liner shipping network (eventually paths), the variables are initialized by pairing the neigbor ports in the services above. Thereby, the master problem is reduced to only respect the strict subset of all feasible patterns i.e. patterns using the generated services.

In [ ]:
W=100 # skibe cap
cardinalityM=5
M=[1:cardinalityM]
A=Matrix(I, cardinalityM, cardinalityM)
p=zeros(5)
b=[45; 38; 25; 11; 12]
w=[22; 42; 52; 53; 78]

# flowModel = Model() # Model for the master problem
# Jprime = 1:size(A, 2) # Initial number of variables
# @variable(flowModel, 0 <= x[Jprime] <= 1000000) # Defining the variables
# @objective(flowModel, Min, sum(x[j] for j in Jprime)) # Setting the objective
# @constraint(flowModel, [i=1:cardinalityM], sum(A[i, j] * x[j] for j in Jprime) == b[i])
# print(flowModel)

flowModel = Model() # Model for the master problem
n_k = size(baltic_demand, 1)
k = 1:size(baltic_demand, 1) # Initial number of variables
A = Matrix(I, n_k, n_k)
@variable(flowModel, x[k] >= 0) # Defining the variables
@objective(flowModel, Min, sum(x[j] for j in k)) # Setting the objective
@constraint(flowModel, [i=1:n_k], sum(A[i, j] * x[j] for j in k) == d_ij[i])
;

J = 1_000  # Some large number
model = Model(HiGHS.Optimizer)
set_silent(model)
@variable(model, x[x_ij] >= 0, Int)
@variable(model, y[getindex.(c_y, 1)] >= 0, Int)
;